In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import linalg as lg
import time
from tqdm import tqdm

from lightonml.projections.sklearn import OPUMap

In [ ]:
def cconj(A):
    # simple conjugate transpose
    return np.conj(A.T)

def do_MDS(D, number_of_anchors):
    # does MDS
    
    m = number_of_anchors
    J = np.eye(m + 2) - 1. / (m + 2) * np.ones((m + 2, m + 2))
    G = -1/2 * np.dot(J, D).dot(J)
    U, s, VT = np.linalg.svd(G)
    Z_est_R2 = np.dot(np.diag(np.sqrt(s[:2])), VT[:2, :])
    Z_est_cpx = Z_est_R2[0, :] + 1j*Z_est_R2[1, :]
    
    # translate the origin back at (0, 0)
    Z_est_cpx -= Z_est_cpx[m + 1]
    
    return Z_est_cpx

def ortho_procrustes(fixed, modify):
    # does the Procrustes analysis following procedure Dokmanic et al. in references
    # and https://en.wikipedia.org/wiki/Orthogonal_Procrustes_problem
    # assumes that the points to align are in column with index 1 onwards.
    # we want to align modify with fixed
    
    fixed = np.vstack ((np.real(fixed[1:]), np.imag(fixed[1:])))
    modify = np.vstack ((np.real(modify), np.imag(modify)))
    original = modify.copy()
    modify = modify[:,1:]
    fixed_mean = (np.mean(fixed, axis=1)).reshape([-1,1])
    fixed -= fixed_mean
    modify_mean = (np.mean(modify, axis=1)).reshape([-1,1])
    modify -= modify_mean
    M = fixed @ modify.T
    u, s, vh = np.linalg.svd(M)
    R = u @ vh
    original = R @ (original - modify_mean @ np.ones(
        [1, original.shape[1]])) + fixed_mean@np.ones([1, original.shape[1]])
    return original[0] + 1j*original[1]

def make_D_ensembles(y, number_of_anchors):
    # populates the distance matrices for all rows
    
    num_elements = int((number_of_anchors+2)* (number_of_anchors+1) * 0.5)
    
    trials = y.shape[1]
    dim = number_of_anchors+2
    all_D_oracles_x = np.zeros([trials, dim, dim])
    
    ind = np.triu_indices(all_D_oracles_x[0].shape[0], k=1)
    for i in range(trials):
        data = y[0:num_elements,i]
        all_D_oracles_x[i][ind] = data
        all_D_oracles_x[i] += all_D_oracles_x[i].T
        
    return all_D_oracles_x

def interfere_with_anchors(n, x, anchors):
    interfered = anchors - x # interfere all anchors with signal
    interfered = np.vstack((interfered, x)) # x with zero (zero is less than x so subtract the other way)
    
    anchors = np.vstack((anchors, np.zeros(n))) # zero is an anchor too
    
    for i in range(anchors.shape[0]-1):
        # subtract all anchors from each other
        diffs = anchors[i] - anchors[1+i:]
        interfered = np.vstack((interfered, diffs))
    
    return interfered

In [ ]:
def get_OPU_measurements(opu_input, num_rand_proj):    
    mapping = OPUMap(n_components=num_rand_proj, verbose_level=2)
    mapping.opu.device.exposure_us = 400 # exposure needs to be chosen so that there is no saturation.
    y = mapping.transform(opu_input.astype('uint8'))
    print (y.shape)
    print (np.max(y))
    print (np.min(y))
    
    return y

def make_anchors(X, number_of_anchors):
    # combine all rows of matrix
    # then start creating anchors which would not be negative when matrix elements are subtracted from it
    X_sum = np.sum(X.copy(), axis=0)
    X_sum[X_sum>0] = 1
    
    anchors = np.zeros([number_of_anchors, n**2]) # store the anchors here
    
    anchor_p = [0.85,0.15]
    
    anchors[0] = np.random.choice([0,1], size=X.shape[1], p=anchor_p) + X_sum # first anchor is random binary
    
    for i in range(1, number_of_anchors):
        # add random binary to previous anchor to get next anchor
        anchors[i] = np.random.choice([0,1], size=X.shape[1], p=anchor_p) + anchors[i-1]

    anchors[anchors>0] = 1 # input cannot be above 1
    anchors = anchors[::-1] # reverse the order for later convenience when interfering with signal
    
    return anchors

def opu_projection(X, k):
    # does random gaussian matrix multiplication using OPU
    # recovers measurement phase too
    
    number_of_anchors = 5
    
    anchors = make_anchors(X, number_of_anchors)

    # add a dummy all-zero signal and intereferes it. Used to localize anchors later
    x = np.zeros([1, X.shape[1]])
    opu_input = interfere_with_anchors(X.shape[1], x, anchors)
    anchors_input_size = opu_input.shape[0]
    
    # interfere each row of X with the anchors
    for i in range(X.shape[0]):
        x = X[i]
        opu_input = np.vstack((opu_input, interfere_with_anchors(X.shape[1], x, anchors)))
    
    num_of_rows_in_A = k
    print('Getting OPU data')
    y_quant = get_OPU_measurements(opu_input, num_of_rows_in_A)
    print('Got OPU data')
    
    # localise anchors using dummy signal
    anchor_positions = np.zeros([num_of_rows_in_A, number_of_anchors+2]).astype('complex128')
    all_D_quant = make_D_ensembles(y_quant[:anchors_input_size], number_of_anchors)
    for i in range(num_of_rows_in_A):
        anchor_positions[i] = do_MDS(all_D_quant[i], number_of_anchors)
        
    # localise other points element by element using our algorithm
    results = np.ones([X.shape[0], num_of_rows_in_A]).astype('complex128')
    for i in range(X.shape[0]):
        all_D_quant = make_D_ensembles(y_quant[(i+1)*anchors_input_size:(i+2)*anchors_input_size], number_of_anchors)
        for row in range(num_of_rows_in_A):
            recovered_points = do_MDS(all_D_quant[row], number_of_anchors)
            recovered_points = ortho_procrustes(anchor_positions[row], recovered_points)
            results[i, row] = recovered_points[0]
    
    return results

def randsvd(M, k):
    # does randomized SVD
    
    m, n = M.shape
    
    Y = opu_projection(M, k)
    Y = cconj(Y)
    
    # split complex projection into two so its equivalent to projection with real matrix
    Y = np.vstack((np.real(Y), np.imag(Y)))
    Y = cconj(Y)
    
    # remaining steps are Halko's randomized SVD algorithm
    Q = lg.orth(Y)
    B = (cconj(Q)) @ M
    U_tilde, s, Vh = lg.svd(B, full_matrices=False)
    U = Q @ U_tilde
    return U, s, Vh

In [ ]:
def trials():
    # vary number of projections and check the error
    num_trials = 10 # reduce this for a faster result
    ks = 10
    
    errors = np.zeros([ks, num_trials])
    
    for k in tqdm(range(1,ks+1)):
        for trial in range(num_trials):
    
            m = 10
            n = 10000
            M = np.random.choice([0,1], size=[m, n], p=[0.8,0.2])
            U, s, Vh = randsvd(M, k=k) # SVD of M
            error = lg.norm(M - U @ np.diag(s) @ Vh) / M.size
            errors[k-1, trial] = error
    
    return M, np.real(U @ np.diag(s) @ Vh), errors

In [ ]:
B, B_rec, errors_ar = trials()

In [ ]:
errors = np.mean(errors_ar, axis=1)
plt.rcParams.update({'font.size': 14})
plt.figure(figsize=(4,3.5))
x_axis = np.linspace(1,10, num=10)
plt.plot(x_axis, errors, linewidth=2.5)
plt.yscale('log')
plt.ylabel('Average error per entry')
plt.xlabel('Number of projections')
plt.xticks(np.arange(min(x_axis), max(x_axis)+1, 2.0)+1)
plt.grid(which='major')
plt.tight_layout()
plt.savefig('svd_OPU_recon_error.pdf')

In [ ]:
from lightonml.datasets import MNIST
from lightonml.encoding.base import BinaryThresholdEncoder

def mnist_matrix():
    # make a matrix of flattened MNIST images
    
    # load data
    (_, _), (X, _) = MNIST()
    # encode data
    encoder = BinaryThresholdEncoder()
    X_encoded = encoder.transform(X)
    
    return X_encoded[:500].reshape([500,-1])

In [ ]:
# randomized SVD on MNIST matrix
M = mnist_matrix()
U, s, Vh = randsvd(M, k=np.min(M.shape))
error = lg.norm(M - U @ np.diag(s) @ Vh) / M.size

In [ ]:
# True SVD for comparison
U_true, s_true, Vh_true = np.linalg.svd(M, full_matrices=False)

In [ ]:
# view results

n_svecs = 7

plt.rcParams.update({'font.size': 12})
plt.figure(figsize=(8.2, 2.75));
for i in range(n_svecs):
    fig = plt.subplot(2,n_svecs, 1+i)
    phase = (Vh[i, 100]/ Vh_true[i, 100]) # rotation of system may be required
    img = np.real(np.conj(phase)*Vh[i]).reshape([28,28])
    plt.imshow(img, cmap='gray')
    plt.xticks([])
    plt.yticks([])
    
    fig = plt.subplot(2,n_svecs, n_svecs + 1+i)
    img_true = (Vh_true[i]).reshape([28,28])
    plt.imshow(img_true, cmap='gray')
    plt.xticks([])
    plt.yticks([])
    rel_error = 100*np.linalg.norm(img-img_true) / np.linalg.norm(img_true)
    plt.title('{:.2e}'.format(rel_error), y=-0.35, fontsize=12)

plt.suptitle('Leading right singular vectors')
plt.figtext(0.0125,0.78,'OPU', size=13, ha='center', rotation='vertical')
plt.figtext(0.0125,0.39,'Python', size=13, ha='center', rotation='vertical')
plt.figtext(0.5,0,'Relative error', size=12, ha='center')
plt.tight_layout()